In [152]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
import pyspark.sql.functions as f
import pyspark.sql.types as t
import os
from functools import reduce
from datetime import datetime, time
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql.functions import lag
from pyspark.sql.window import Window
from pyspark.sql.functions import pandas_udf
from pyspark.sql.types import DoubleType
import pandas as pd
import math
import folium

In [153]:
spark = SparkSession.builder.master('local[*]') \
    .appName('Métricas') \
    .getOrCreate()

<p>1 – Quantidade de ônibus em operação</p>

In [154]:
directory_path = "/home/rodrigo/projetos/monitoramento_sptrans/data/datalake/prata/operacao_agrupada/"
file_list = os.listdir(directory_path)
dataframes_agrupados = []
for item in file_list:
    if item.endswith('.parquet'):
        caminho_completo = os.path.join(directory_path, item)

        df = spark.read.parquet(caminho_completo)
        dataframes_agrupados.append(df)



In [155]:
def union_all(dfs):
    return reduce(DataFrame.unionAll, dfs)

In [156]:
dataframes_agrupados_completo = union_all(dataframes_agrupados)
dataframes_agrupados_completo.show(truncate=False)

+-------+-----------+-------------------------------------------+-------------------------------------------+--------+-------------+-----------------+--------------------+-----------------+---------------------+----------------------+-----------------+
|LINHA  |CODIGO_AREA|CONSOCIO                                   |EMPRESA                                    |HORA_API|DATA_EXTRACAO|LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|LETREIRO_ORIGEM  |LETREIRO_DESTINO     |QTDE_VEICULOS_OPERACAO|DATA_EXTRACAO_API|
+-------+-----------+-------------------------------------------+-------------------------------------------+--------+-------------+-----------------+--------------------+-----------------+---------------------+----------------------+-----------------+
|3054-10|4          |PÊSSEGO TRANSPORTES LTDA                   |PÊSSEGO TRANSPORTES LTDA                   |08:50   |2023-09-15   |3054-10          |33743               |HOSP. SAPOPEMBA  |JD. PALANQUE         |6                     |2023-09

In [157]:
dataframes_agrupados_completo.printSchema()

root
 |-- LINHA: string (nullable = true)
 |-- CODIGO_AREA: string (nullable = true)
 |-- CONSOCIO: string (nullable = true)
 |-- EMPRESA: string (nullable = true)
 |-- HORA_API: string (nullable = true)
 |-- DATA_EXTRACAO: date (nullable = true)
 |-- LETREIRO_COMPLETO: string (nullable = true)
 |-- CODIGO_IDENTIFICADOR: long (nullable = true)
 |-- LETREIRO_ORIGEM: string (nullable = true)
 |-- LETREIRO_DESTINO: string (nullable = true)
 |-- QTDE_VEICULOS_OPERACAO: long (nullable = true)
 |-- DATA_EXTRACAO_API: date (nullable = true)



In [158]:
dataframes_agrupados_completo = dataframes_agrupados_completo.drop('DATA_EXTRACAO_API')
dataframes_agrupados_completo.show()

+-------+-----------+--------------------+--------------------+--------+-------------+-----------------+--------------------+-----------------+--------------------+----------------------+
|  LINHA|CODIGO_AREA|            CONSOCIO|             EMPRESA|HORA_API|DATA_EXTRACAO|LETREIRO_COMPLETO|CODIGO_IDENTIFICADOR|  LETREIRO_ORIGEM|    LETREIRO_DESTINO|QTDE_VEICULOS_OPERACAO|
+-------+-----------+--------------------+--------------------+--------+-------------+-----------------+--------------------+-----------------+--------------------+----------------------+
|3054-10|          4|PÊSSEGO TRANSPORT...|PÊSSEGO TRANSPORT...|   08:50|   2023-09-15|          3054-10|               33743|  HOSP. SAPOPEMBA|        JD. PALANQUE|                     6|
|6026-10|          6|TRANSWOLFF TRANSP...|TRANSWOLFF TRANSP...|   08:50|   2023-09-15|          6026-10|                  87| TERM. STO. AMARO|          JD. ICARAÍ|                     5|
|857A-10|          7|      CONSÓRCIO KBPX|KBPX ADMINISTRAÇÃ.

In [159]:
dataframes_agrupados_completo.printSchema()

root
 |-- LINHA: string (nullable = true)
 |-- CODIGO_AREA: string (nullable = true)
 |-- CONSOCIO: string (nullable = true)
 |-- EMPRESA: string (nullable = true)
 |-- HORA_API: string (nullable = true)
 |-- DATA_EXTRACAO: date (nullable = true)
 |-- LETREIRO_COMPLETO: string (nullable = true)
 |-- CODIGO_IDENTIFICADOR: long (nullable = true)
 |-- LETREIRO_ORIGEM: string (nullable = true)
 |-- LETREIRO_DESTINO: string (nullable = true)
 |-- QTDE_VEICULOS_OPERACAO: long (nullable = true)



In [162]:
@f.udf(returnType=t.StringType())
def turno(hora: str):
    hora_formatada = datetime.strptime(hora, '%H:%M').time()
    if 0 <= hora_formatada.hour < 6:
        return 'Madrugada'
    elif 6 <= hora_formatada.hour < 12:
        return 'Manhã'
    elif 12 <= hora_formatada.hour < 18:
        return 'Tarde'
    else:
        return 'Noite'
        

In [163]:
turno('23:55')

Column<'turno(23:55)'>

In [164]:
dataframes_agrupados_completo = dataframes_agrupados_completo.withColumn('TURNO', turno(f.col('HORA_API'))) 

df_filter = dataframes_agrupados_completo.select(
    dataframes_agrupados_completo.CONSOCIO,
    dataframes_agrupados_completo.TURNO,
    dataframes_agrupados_completo.DATA_EXTRACAO ,
    dataframes_agrupados_completo.LETREIRO_ORIGEM,
    dataframes_agrupados_completo.LETREIRO_DESTINO,
    dataframes_agrupados_completo.HORA_API,
    dataframes_agrupados_completo.LETREIRO_COMPLETO,
    dataframes_agrupados_completo.QTDE_VEICULOS_OPERACAO,
    dataframes_agrupados_completo.CODIGO_IDENTIFICADOR,
    ).filter(
        (dataframes_agrupados_completo.DATA_EXTRACAO == '2023-09-15' ) & 
        (dataframes_agrupados_completo.TURNO == 'Noite') &
        (dataframes_agrupados_completo.LETREIRO_COMPLETO == '9047-10')
        ) \
        
df_filter.show(truncate=False)

+-----------------------+-----+-------------+---------------+----------------+--------+-----------------+----------------------+--------------------+
|CONSOCIO               |TURNO|DATA_EXTRACAO|LETREIRO_ORIGEM|LETREIRO_DESTINO|HORA_API|LETREIRO_COMPLETO|QTDE_VEICULOS_OPERACAO|CODIGO_IDENTIFICADOR|
+-----------------------+-----+-------------+---------------+----------------+--------+-----------------+----------------------+--------------------+
|CONSÓRCIO TRANSNOROESTE|Noite|2023-09-15   |LAPA           |JD. PAULISTANO  |18:00   |9047-10          |9                     |33651               |
|CONSÓRCIO TRANSNOROESTE|Noite|2023-09-15   |LAPA           |JD. PAULISTANO  |18:00   |9047-10          |5                     |883                 |
|CONSÓRCIO TRANSNOROESTE|Noite|2023-09-15   |LAPA           |JD. PAULISTANO  |18:05   |9047-10          |7                     |33651               |
|CONSÓRCIO TRANSNOROESTE|Noite|2023-09-15   |LAPA           |JD. PAULISTANO  |18:05   |9047-10      

In [165]:
df_filter.groupBy(['CONSOCIO', 'LETREIRO_COMPLETO',  'LETREIRO_ORIGEM', 'LETREIRO_DESTINO','HORA_API', 'TURNO']) \
    .agg(
        F.sum('QTDE_VEICULOS_OPERACAO') \
        .alias('QUANTIDADE_VEICULOS_OPERACAO')
    ) \
    .orderBy('HORA_API')\
    .show(truncate=False)

+-----------------------+-----------------+---------------+----------------+--------+-----+----------------------------+
|CONSOCIO               |LETREIRO_COMPLETO|LETREIRO_ORIGEM|LETREIRO_DESTINO|HORA_API|TURNO|QUANTIDADE_VEICULOS_OPERACAO|
+-----------------------+-----------------+---------------+----------------+--------+-----+----------------------------+
|CONSÓRCIO TRANSNOROESTE|9047-10          |LAPA           |JD. PAULISTANO  |18:00   |Noite|14                          |
|CONSÓRCIO TRANSNOROESTE|9047-10          |LAPA           |JD. PAULISTANO  |18:05   |Noite|13                          |
|CONSÓRCIO TRANSNOROESTE|9047-10          |LAPA           |JD. PAULISTANO  |18:10   |Noite|15                          |
|CONSÓRCIO TRANSNOROESTE|9047-10          |LAPA           |JD. PAULISTANO  |18:15   |Noite|14                          |
|CONSÓRCIO TRANSNOROESTE|9047-10          |LAPA           |JD. PAULISTANO  |18:20   |Noite|15                          |
|CONSÓRCIO TRANSNOROESTE|9047-10

In [166]:
directory_path = "/home/rodrigo/projetos/monitoramento_sptrans/data/datalake/prata/operacao_desagrupada"
file_list = os.listdir(directory_path)
dataframes_desagrupados = []
for item in file_list:
    if item.endswith('.parquet'):
        caminho_completo = os.path.join(directory_path, item)

        df = spark.read.parquet(caminho_completo)
        dataframes_desagrupados.append(df)



In [167]:
dataframes_desagrupados_completo = union_all(dataframes_desagrupados)
dataframes_desagrupados_completo = dataframes_desagrupados_completo.drop('LINHA')
dataframes_desagrupados_completo = dataframes_desagrupados_completo.withColumn('TURNO', turno(f.col('HORA_API')))
dataframes_desagrupados_completo.show(truncate=False)

+-----------+-------------------------------------------+-------------------------------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|CODIGO_AREA|CONSOCIO                                   |EMPRESA                                    |DATA_EXTRACAO|HORA_API|LETREIRO_COMPLETO|SENTIDO_OPERACAO|CODIGO_IDENTIFICADOR_LINHA|PREFIXO_ONIBUS|DATA_HORA_CAPTURA_LOCALIZACAO|LATITUDE           |LONGITUDE          |DATA_EXTRACAO_API|TURNO|
+-----------+-------------------------------------------+-------------------------------------------+-------------+--------+-----------------+----------------+--------------------------+--------------+-----------------------------+-------------------+-------------------+-----------------+-----+
|4          |PÊSSEGO TRANSPORTES LTDA                   |PÊSSEGO TRANSPORTES LTDA                   |2023-09-15 

In [168]:
dataframe_prefixo_onibus = dataframes_desagrupados_completo \
    .select(
        dataframes_desagrupados_completo.HORA_API,
        dataframes_desagrupados_completo.PREFIXO_ONIBUS,
        dataframes_desagrupados_completo.CODIGO_IDENTIFICADOR_LINHA,
        dataframes_desagrupados_completo.LETREIRO_COMPLETO,
        dataframes_desagrupados_completo.LATITUDE,
        dataframes_desagrupados_completo.LONGITUDE,
    ) \
    .filter(
        (dataframes_desagrupados_completo.PREFIXO_ONIBUS == '55393') &
        (dataframes_desagrupados_completo.TURNO == 'Noite') &
        (dataframes_desagrupados_completo.DATA_EXTRACAO == '2023-09-18')
    )



dataframe_prefixo_onibus.show(33)

+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+
|   18:00|         55393|                     34448|          4726-10|        -23.5661365|        -46.5800915|
|   18:05|         55393|                     34448|          4726-10|-23.562075749999998|         -46.592059|
|   18:10|         55393|                     34448|          4726-10|        -23.5633635|-46.591837500000004|
|   18:15|         55393|                     34448|          4726-10|        -23.5633635|-46.591837500000004|
|   18:20|         55393|                     34448|          4726-10|        -23.5633635|-46.591837500000004|
|   18:25|         55393|                     34448|          4726-10|-23.563520500000003|        -46.5915385|
|

In [169]:
def haversine(lat1, lon1, lat2, lon2):

    r = 6371.0

    lat1 = math.radians(lat1)
    lon1 = math.radians(lon1)
    lat2 = math.radians(lat2)
    lon2 = math.radians(lon2)

    dlat = lat2 - lat1
    dlon = lon2 - lon1

 
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))

    distance = r * c

    return distance

@pandas_udf(DoubleType())
def haversine_udf(lat, lon):
    distances = []
    for i in range(len(lat)):
        
        if i == 0:
            distances.append(0.0)  
        else:
            distance = haversine(lat[i], lon[i], lat[i - 1], lon[i - 1])
            distances.append(distance)
    return pd.Series(distances)

df= dataframe_prefixo_onibus.withColumn("DISTANCIA", haversine_udf(dataframe_prefixo_onibus["LATITUDE"], dataframe_prefixo_onibus["LONGITUDE"]))


df.show(33)


soma_distancias = df.selectExpr("sum(DISTANCIA)").collect()[0][0]
print(f'Soma das distâncias: {soma_distancias:.2f} km')



+--------+--------------+--------------------------+-----------------+-------------------+-------------------+-------------------+
|HORA_API|PREFIXO_ONIBUS|CODIGO_IDENTIFICADOR_LINHA|LETREIRO_COMPLETO|           LATITUDE|          LONGITUDE|          DISTANCIA|
+--------+--------------+--------------------------+-----------------+-------------------+-------------------+-------------------+
|   18:00|         55393|                     34448|          4726-10|        -23.5661365|        -46.5800915|                0.0|
|   18:05|         55393|                     34448|          4726-10|-23.562075749999998|         -46.592059| 1.3006534951896462|
|   18:10|         55393|                     34448|          4726-10|        -23.5633635|-46.591837500000004|0.14496006505067272|
|   18:15|         55393|                     34448|          4726-10|        -23.5633635|-46.591837500000004|                0.0|
|   18:20|         55393|                     34448|          4726-10|        -23.5

Soma das distâncias: 14.60 km


In [170]:
lista_de_pontos = dataframe_prefixo_onibus.rdd.flatMap(lambda linha : [[linha.LATITUDE,linha.LONGITUDE] ]).collect()
lista_de_pontos[int(len(lista_de_pontos) // 2)] 

[-23.5410385, -46.575868]

In [171]:
lista_de_pontos

[[-23.5661365, -46.5800915],
 [-23.562075749999998, -46.592059],
 [-23.5633635, -46.591837500000004],
 [-23.5633635, -46.591837500000004],
 [-23.5633635, -46.591837500000004],
 [-23.563520500000003, -46.5915385],
 [-23.56809825, -46.580418],
 [-23.561913, -46.57990375],
 [-23.555286000000002, -46.5844125],
 [-23.5441195, -46.5774155],
 [-23.5409625, -46.5764615],
 [-23.541229, -46.5766665],
 [-23.5410385, -46.575868],
 [-23.5410385, -46.575868],
 [-23.5410385, -46.575868],
 [-23.543112999999998, -46.580823],
 [-23.553994, -46.583692],
 [-23.5585915, -46.586188500000006],
 [-23.563505499999998, -46.5787415],
 [-23.5622935, -46.5916045],
 [-23.563520500000003, -46.5915385],
 [-23.563520500000003, -46.5915385],
 [-23.56406775, -46.587662],
 [-23.561881, -46.58030525],
 [-23.549926, -46.580749],
 [-23.54132, -46.57769475],
 [-23.541, -46.576643000000004],
 [-23.541, -46.576643000000004]]

In [172]:
dataframe_prefixo_onibus.select(
    dataframe_prefixo_onibus.HORA_API
).distinct().show()

+--------+
|HORA_API|
+--------+
|   18:05|
|   18:32|
|   19:15|
|   19:10|
|   19:35|
|   20:00|
|   18:40|
|   18:15|
|   19:05|
|   18:35|
|   19:20|
|   20:15|
|   20:20|
|   18:10|
|   18:45|
|   19:00|
|   19:55|
|   19:50|
|   18:00|
|   19:30|
+--------+
only showing top 20 rows



In [173]:
m = folium.Map(
    location=lista_de_pontos[int(len(lista_de_pontos) // 2)] ,
    zoom_start=10,
    control_scale=True)
for ponto in lista_de_pontos:
   folium.Marker(
    location=ponto,
    popup='Teste',
    icon=folium.Icon(
        color='red', 
        icon='icon-sigin'
    )
).add_to(m)
m


In [174]:
posicao = [[-23.563553, -46.591361],
 [-23.563457, -46.591302],
 [-23.563153, -46.591127],
 [-23.563273, -46.591021],
 [-23.562679, -46.590712],
 [-23.563173, -46.589637],
 [-23.563546, -46.588816],
 [-23.563753, -46.588345],
 [-23.563970, -46.587874],
 [-23.564361, -46.587026],
 [-23.564402, -46.586923],
]

In [175]:
df_trajeto_linhas = spark.read.options(delimiter=',', header=True, inferSchema='True').csv('../data/datalake/bronze/arquivos_gtfs/shapes.txt', )

df_trajeto_linhas.printSchema()

root
 |-- shape_id: integer (nullable = true)
 |-- shape_pt_lat: double (nullable = true)
 |-- shape_pt_lon: double (nullable = true)
 |-- shape_pt_sequence: integer (nullable = true)
 |-- shape_dist_traveled: double (nullable = true)



In [179]:
from pyspark.sql.functions import col
from pyspark.sql.types import FloatType

In [186]:
df_trajeto_linhas = spark.read.options(delimiter=',', header=True).csv('../data/datalake/bronze/arquivos_gtfs/shapes.txt', )
df_linha_filter = df_trajeto_linhas.filter(df_trajeto_linhas.shape_id.isin(['63239', '73115']) )
df_linha_filter = df_linha_filter.withColumn('shape_pt_lat', col('shape_pt_lat').cast(FloatType()))
df_linha_filter = df_linha_filter.withColumn('shape_pt_lon', col('shape_pt_lon').cast(FloatType()))
posicao = df_linha_filter.rdd.flatMap(lambda linha: [[linha.shape_pt_lat, linha.shape_pt_lon]]).collect()
posicao
# df_linha_filter.show()

[[-23.563552856445312, -46.59136199951172],
 [-23.563457489013672, -46.59130096435547],
 [-23.563152313232422, -46.59112548828125],
 [-23.56327247619629, -46.59102249145508],
 [-23.562679290771484, -46.59071350097656],
 [-23.563173294067383, -46.589637756347656],
 [-23.56354522705078, -46.58881759643555],
 [-23.563753128051758, -46.58834457397461],
 [-23.5639705657959, -46.58787536621094],
 [-23.564361572265625, -46.5870246887207],
 [-23.564401626586914, -46.58692169189453],
 [-23.564531326293945, -46.586639404296875],
 [-23.564611434936523, -46.586463928222656],
 [-23.564966201782227, -46.58568572998047],
 [-23.56502914428711, -46.585514068603516],
 [-23.56557846069336, -46.585567474365234],
 [-23.565752029418945, -46.58558654785156],
 [-23.566030502319336, -46.58560562133789],
 [-23.566450119018555, -46.58563995361328],
 [-23.56669044494629, -46.585662841796875],
 [-23.566835403442383, -46.58567428588867],
 [-23.567049026489258, -46.5857048034668],
 [-23.567276000976562, -46.58571243

In [187]:
mapa_linhas = folium.Map(location = [-23.563553, -46.591361], 
                         zoom_start = 11, 
                         control_scale = True, 
                         tiles = "cartodbpositron")



for i in range(len(posicao)-1):
    folium.PolyLine(locations = [[posicao[i][0], posicao[i][1]], [posicao[i+1][0], posicao[i+1][1]]],
                   color = 'navy').add_to(mapa_linhas)
    
mapa_linhas.add_child(folium.LatLngPopup())

mapa_linhas